In [1]:
import sqlite3, config
import pandas as pd

In [2]:
conn = sqlite3.connect(config.DB_FILE)
cursor = conn.cursor()

In [3]:
q = """
    SELECT *
    FROM stock
    LIMIT 20
    """

df = pd.read_sql(q, conn)
df.head()

,id,symbol,name,exchange,shortable,backtest_data
0,1,ARAY,Accuray Incorporated Common Stock,NASDAQ,1,0
1,2,ARKG,ARK Genomic Revolution ETF,BATS,1,0
2,3,ARKQ,ARK Autonomous Technology & Robotics ETF,BATS,1,0
3,4,CBAT,"CBAK Energy Technology, Inc. Common Stock",NASDAQ,0,0
4,5,CBLI,"Cleveland BioLabs, Inc. Common Stock",NASDAQ,0,0


## Adding backtest_data column to stock table

Adding a backtest_data column to the stock table. It will be a simple boolean column.

This will be used to populate a list of available stocks that can be used when testing strategies on the backtesting page.

In [4]:
# cursor.execute("""
#     ALTER TABLE stock
#     ADD backtest_data BOOLEAN NOT NULL
#     DEFAULT 0;
# """)

confirm 

In [5]:
df = pd.read_sql(q, conn)
df.head()

,id,symbol,name,exchange,shortable,backtest_data
0,1,ARAY,Accuray Incorporated Common Stock,NASDAQ,1,0
1,2,ARKG,ARK Genomic Revolution ETF,BATS,1,0
2,3,ARKQ,ARK Autonomous Technology & Robotics ETF,BATS,1,0
3,4,CBAT,"CBAK Energy Technology, Inc. Common Stock",NASDAQ,0,0
4,5,CBLI,"Cleveland BioLabs, Inc. Common Stock",NASDAQ,0,0


### Update backtest_data 

Eventually will add this update to the populate_minute_date script

In [6]:
q = """
    SELECT DISTINCT stock_id
    FROM stock_price_minute
    """

df = pd.read_sql(q, conn)
df.head()

,stock_id
0,1311
1,112
2,93
3,7368
4,6590


In [8]:
backtest_stocks = df.stock_id.drop_duplicates().tolist()
len(backtest_stocks)

103

In [9]:
backtest_stocks = tuple(backtest_stocks)

In [10]:
q = """
    SELECT id, name
    FROM stock
    WHERE id IN {}
    """.format(backtest_stocks)

df = pd.read_sql(q, conn)
df.head()

,id,name
0,8,"DocuSign, Inc. Common Stock"
1,10,"Facebook, Inc. Class A Common Stock"
2,23,Booking Holdings Inc. Common Stock
3,93,"Advanced Micro Devices, Inc. Common Stock"
4,112,Adobe Inc. Common Stock


UPDATE backtest_data column:

In [11]:
cursor.execute("""
    UPDATE stock
    SET backtest_data = 1
    WHERE id IN {}
""".format(backtest_stocks))

In [12]:
conn.commit()

confirm

In [13]:
q = """
    SELECT id, name, backtest_data
    FROM stock
    WHERE backtest_data = 1
    """

df = pd.read_sql(q, conn)
df.head(20)

,id,name,backtest_data
0,8,"DocuSign, Inc. Common Stock",1
1,10,"Facebook, Inc. Class A Common Stock",1
2,23,Booking Holdings Inc. Common Stock,1
3,93,"Advanced Micro Devices, Inc. Common Stock",1
4,112,Adobe Inc. Common Stock,1
5,115,"Amazon.com, Inc. Common Stock",1
6,309,"Analog Devices, Inc. Common Stock",1
7,316,BioMarin Pharmaceutical Inc. Common Stock,1
8,565,"Automatic Data Processing, Inc. Common Stock",1
9,592,CDW Corporation Common Stock,1


## Create backtest_config table

In [3]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS backtest_config (
        run_id INTEGER PRIMARY KEY,
        date NOT NULL,
        stock_id INTEGER,
        strategy NOT NULL,
        bt_start NOT NULL,
        bt_end NOT NULL,
        set_cash INTEGER NOT NULL,
        open_range INTEGER NOT NULL,
        liquidate_time INTEGER NOT NULL,
        FOREIGN KEY (stock_id) REFERENCES stock (id)
    )
""")

In [4]:
conn.commit()

In [5]:
q = """
    SELECT *
    FROM backtest_config
    """

df = pd.read_sql(q, conn)
df.head(20)

,run_id,date,stock_id,strategy,bt_start,bt_end,set_cash,open_range,liquidate_time
